<a href="https://colab.research.google.com/github/akm-2016/Emergency-Vehicle-Or-Not/blob/master/EmergencyVehicle_CV_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing libraries

import io
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import losses, optimizers, metrics

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
!unzip train_SOaYf6m.zip

In [0]:
train = pd.read_csv('train.csv')

In [36]:
train_image = []

for i in tqdm(range(train.shape[0])):
    img = image.load_img('images/'+train['image_names'][i], target_size=(224,224), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)

y=train['emergency_or_not'].values
y = keras.utils.np_utils.to_categorical(y)

100%|██████████| 1646/1646 [00:02<00:00, 662.74it/s]


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [38]:
print(X_train.shape, y_train.shape)

(1316, 224, 224, 3) (1316, 2)


In [39]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [40]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 1026      
Total params: 27,561,282
Trainable params: 27,561,282
Non-trainable params: 0
_________________________________________________________________


In [47]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.Adam(lr=5e-5), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

history = model.fit(X_train, y_train,
                              # steps_per_epoch=300, 
                              epochs=10, batch_size=32,
                              validation_data=(X_test, y_test),
                              # validation_steps=12
                    )

This is the number of trainable weights before freezing the conv base: 4
Train on 1316 samples, validate on 330 samples
Epoch 1/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.2542 - accuracy: 0.9027 - val_loss: 0.2373 - val_accuracy: 0.9091
Epoch 2/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.1190 - accuracy: 0.9681 - val_loss: 0.2114 - val_accuracy: 0.9121
Epoch 3/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.0683 - accuracy: 0.9863 - val_loss: 0.2081 - val_accuracy: 0.9152
Epoch 4/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.0633 - accuracy: 0.9924 - val_loss: 0.2044 - val_accuracy: 0.9303
Epoch 5/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.0405 - accuracy: 0.9939 - val_loss: 0.2151 - val_accuracy: 0.9212
Epoch 6/10
1316/1316 [==============================] - 16s 12ms/step - loss: 0.0230 - accuracy: 0.9970 - val_loss: 0.2150 - val_accuracy: 0.9182
Epoc

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
from keras.models import load_model
try:
    model = load_model('checkpoints\\model.h5')
except:
    pass

In [51]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit(X_train, y_train,
                              #steps_per_epoch=300, 
                              epochs=10, batch_size=32,
                               callbacks=[
                                  keras.callbacks.ModelCheckpoint('checkpoints\\model.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1),
                                  keras.callbacks.TensorBoard(log_dir='logs\\inception', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None),
                              ],
                              validation_data=(X_test, y_test), 
                              #validation_steps=12
                    )

/usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v2.py:92: UserWarning: The TensorBoard callback `batch_size` argument (for histogram computation) is deprecated with TensorFlow 2.0. It will be ignored.
  warnings.warn('The TensorBoard callback `batch_size` argument '


Train on 1316 samples, validate on 330 samples
Epoch 1/10
1316/1316 [==============================] - 23s 17ms/step - loss: 0.0135 - categorical_accuracy: 0.9977 - val_loss: 0.2741 - val_categorical_accuracy: 0.9182
Epoch 2/10
1316/1316 [==============================] - 22s 17ms/step - loss: 0.0087 - categorical_accuracy: 0.9977 - val_loss: 0.2605 - val_categorical_accuracy: 0.9333
Epoch 3/10
1316/1316 [==============================] - 22s 17ms/step - loss: 0.0074 - categorical_accuracy: 0.9977 - val_loss: 0.2498 - val_categorical_accuracy: 0.9303
Epoch 4/10
1316/1316 [==============================] - 22s 17ms/step - loss: 0.0136 - categorical_accuracy: 0.9962 - val_loss: 0.2699 - val_categorical_accuracy: 0.9273
Epoch 5/10
1316/1316 [==============================] - 22s 17ms/step - loss: 0.0124 - categorical_accuracy: 0.9985 - val_loss: 0.2432 - val_categorical_accuracy: 0.9303
Epoch 6/10
1316/1316 [==============================] - 22s 17ms/step - loss: 0.0046 - categorical_accu

In [0]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
test = pd.read_csv('test_EV.csv')

In [56]:
test_image = []

for i in tqdm(range(test.shape[0])):
    img = image.load_img('images/'+test['image_names'][i], target_size=(224,224), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)

test = np.array(test_image)

100%|██████████| 706/706 [00:01<00:00, 669.59it/s]


In [0]:
prediction = model.predict_classes(test)

In [0]:
# creating submission file
sample = pd.read_csv('sample_submission_yxjOnvz.csv')
sample['emergency_or_not'] = prediction
sample.to_csv('sample_vgg16.csv', header=True, index=False)